In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [ ]:
summary = pd.read_csv("garmin_edge_820/summary.csv")   
details = pd.read_csv("garmin_edge_820/4557226804_ACTIVITY_data.csv", sep=";")

In [ ]:
details.rename(columns=lambda x: x.replace('record.', ''), inplace=True)
details['time'] = pd.to_datetime(details.pop('timestamp[s]'), unit='s').dt.time
details.set_index("time", inplace=True)

In [ ]:
hr_zones = [(1, 128), (129, 146), (147, 156), (157, 165), (166, 174), (175, 179), (180, float('inf'))]
power_zones = [(1, 157), (158, 186), (187, 200), (201, 218), (219, 247), (248, 287), (281, float('inf'))]
# funzione che calcola le zone di frequenza cardiaca dato il battito in input

def get_zone(hr, zones):
    for zone, (lower, upper) in enumerate(zones, start=1):
        if lower <= hr <= upper:
            return zone

In [ ]:
details['hr_zone'] = details['heart_rate[bpm]'].apply(get_zone, zones=hr_zones)
details['pwr_zone'] = details['power[watts]'].apply(get_zone, zones=power_zones)

In [ ]:
details['time_since_start'] = 1
details['time_since_start'] = details['time_since_start'].cumsum().sub(1)

In [ ]:
def remove_content_after_second_dot(string):
    first_dot_index = string.find('.')  # Trova l'indice del primo punto
    second_dot_index = string.find('.', first_dot_index + 1)  # Trova l'indice del secondo punto
    if second_dot_index != -1:  # Verifica se è presente un secondo punto
        return string[:second_dot_index]  # Restituisce la stringa fino al secondo punto
    else:
        return string  # Se non è presente un secondo punto, restituisce la stringa originale

In [ ]:
details['altitude[m]'] = details['altitude[m]'].apply(remove_content_after_second_dot).astype(float)


# Calcola la differenza di altitudine tra le righe adiacenti
details['altitude_diff'] = details['altitude[m]'].shift(1) - details['altitude[m]'] 

# Calcola la percentuale di pendenza
details['slope_percent'] = details['altitude_diff'] / details['distance[m]'] * 100


In [ ]:
details.loc[details['time_since_start'] > 2000]

In [ ]:
details.plot.scatter(x='time_since_start', y='altitude_diff')

In [ ]:
# Calcola i secondi passati dall'inizio dell'allenamento
start_time = details.index[0]
details['time_since_start'] = (details.index - start_time).total_seconds()

In [ ]:
details.describe()

In [ ]:
details.isna().sum()

### Data Visualization

In [ ]:
details.plot.scatter(y="cadence[rpm]", x="heart_rate[bpm]")